In [1]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import GoogleMapsGeocoding as gmaps
import importlib
import LLAMAconnect
import subprocess
import shutil
from pprint import pprint
import random
import vehParameters

In [2]:
net = sumolib.net.readNet('osm.net.xml')
parkingAreas = list(sumolib.output.parse('osm_stops.add.xml', "parkingArea"))

In [3]:
## Script for generating alternative routes
duaiterate_path = "/usr/share/sumo/tools/assign/duaIterate.py"
def getArgs(net_path, trips_path, additional_path, iterations):
    args = [
        "-n", net_path,
        "-t", trips_path,
        "--additional", additional_path,
        "duarouter--additional-files", additional_path,
        "-l", str(iterations)
    ]
    return args

In [5]:
radius = 150

In [6]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [7]:
def getClosestEdges(lat, lon, radius, maxEdges=10):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])

        ## Checking if the edge found can be used by passenger car
        for dist, edge in distanceAndEdges:
            if edge.allows('passenger'):
                closestEdges.append(edge)

    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}')
    return closestEdges

In [8]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot
    print("No parking spot found:", lat, lon)
    return None

In [9]:
def getPath(location_time_list, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    locations = list(location_time_list.values())
    home = getClosestEdges(*locations[0], radius)[0].getID()
    path.append(home)
    
    for i in range(1, len(locations)-1):
        stop_durations.append(100 * (departures[i] - departures[i - 1]))
        path.append(getParkingSpot(*locations[i], radius, parkingAreas))
    
    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [10]:
def pathToXML(path, vehicleID, veh_type, departure_time, stop_durations):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" type="{veh_type}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'
    xml += '</trip>'
    return xml

Getting the routine for the students described

In [40]:
places ={
    'leisure': ["bar", "movie_theater", "shopping_mall"],
    'eating': ["restaurant", "bakery", "cafe"],
    'shopping': ["store", "drugstore", "supermarket"],
    'sports': ["gym"],
    'institute': ["IB", "IG"],
    'university': ["Unicamp"],
}

student_info = "Today, the student is going to have lunch at a restaurant and the student is going to the supermarket."
importlib.reload(LLAMAconnect)
responses = LLAMAconnect.generate_response_trips(student_info, places, number_of_trips=5) # number of trips defaults to 5
json_response = json.loads(responses[0])
pprint(json_response)

100%|██████████| 5/5 [00:03<00:00,  1.52it/s]

{'10': {'activity': 'study', 'location': 'IB'},
 '11': {'activity': 'study', 'location': 'IB'},
 '12': {'activity': 'lunch', 'location': 'restaurant'},
 '13': {'activity': 'lunch', 'location': 'restaurant'},
 '14': {'activity': 'study', 'location': 'IB'},
 '15': {'activity': 'study', 'location': 'IB'},
 '16': {'activity': 'study', 'location': 'IB'},
 '17': {'activity': 'study', 'location': 'IB'},
 '18': {'activity': 'shopping', 'location': 'supermarket'},
 '19': {'activity': 'shopping', 'location': 'supermarket'},
 '20': {'activity': 'sport', 'location': 'IG'},
 '21': {'activity': 'study', 'location': 'IB'},
 '22': {'activity': 'study', 'location': 'IB'},
 '23': {'activity': 'dinner', 'location': 'home'},
 '7': {'activity': 'wake up', 'location': 'home'},
 '8': {'activity': 'study', 'location': 'IB'},
 '9': {'activity': 'study', 'location': 'IB'}}


Generating different houses for students based on a random number generator in the where students might live

In [12]:
def getHome():
    # top right -22.815539, -47.076272
    # top left: -22.818517, -47.079520
    # bottom left: -22.826410, -47.072591
    # bottom right: -22.823439, -47.068828
    X_MIN = 22815539
    X_MAX = 22826410
    Y_MIN = 47068828
    Y_MAX = 47079520
    while True:
        random_x = random.randint(X_MIN, X_MAX)
        random_y = random.randint(Y_MIN, Y_MAX)
        lat = - random_x / 10**6
        lon = - random_y / 10**6
        if (lat, lon) in coords.values():
            continue
        break
    return lat, lon

In [32]:
def getCoords(trip, sulfixo, institutes, radius, siglas=None):
    # Returns a dictionary with the latitude and longitude of the locations of interest
    # The suffix is the name of the state, city and neighborhood that will be added to the end of each location to improve the search
    # 'siglas' is a dictionary that maps the name of the location to a shorter name, if needed
    coords = {}
    importlib.reload(gmaps)
    for i in range(len(trip)):
        local = trip[f'{i + 7}']['location']
        local_comp = local + ", " + sulfixo

        if local in coords.keys():
            continue
        
        if local == 'home':
            coords['home'] = (-22.821026, -47.072902)
            continue
        elif local in institutes:
                result = gmaps.geocode_address(local_comp)
                lat, lon = result[0]['latitude'], result[0]['longitude']
        else:
            previous = coords[trip[f'{i + 6}']['location']]
            result = gmaps.find_nearby_building(previous[0], previous[1], local, radius=radius)
            while result is False:
                print(f"Could not find {local} in a radius of {radius} meters. Trying again with a larger radius.")
                radius += 100
                result = gmaps.find_nearby_building(previous[0], previous[1], local, radius=radius)
            lat, lon = result[0]['latitude'], result[0]['longitude']
        
        if siglas is not None:
            try:
                coords[f'{siglas[local]}'] = (lat, lon)
            except KeyError:
                coords[f'{local}'] = (lat, lon)
        else:
            coords[f'{local}'] = (lat, lon)
        
    return coords

In [34]:
def coordsToTrip(trip, coords):
    location_time = {}
    home = coords['home'] # Gets a random home location for each student
    last = home
    location_time[7] = last
    
    for j in range(1, len(trip)):
        local_coords = coords[trip[f'{j + 7}']['location']]
        if local_coords != last:
            location_time[j + 7] = local_coords
            last = local_coords

    return location_time

Getting the latitude and longitude for each of the locations

In [41]:
trips = {}
location_time_list = []
for i in range(len(responses)):
    json_response = json.loads(responses[i])
    trips[i] = json_response
    coords = getCoords(trips[i], "Barão Geraldo, Campinas, SP, Brazil", places['institute'], 200)
    location_time_list.append(coordsToTrip(trips[i], coords))


Could not find supermarket in a radius of 200 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 300 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 400 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 500 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 600 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 700 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 800 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 900 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 200 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 300 meters. Trying again with a larger radius.
Could not find supermarket in a radius of 400 meters. Trying again with a larger radius.
Could not find superm

In [42]:
pprint(location_time_list)

[{7: (-22.821026, -47.072902),
  8: (-22.8193885, -47.06952769999999),
  12: (-22.8186891, -47.0695881),
  14: (-22.8193885, -47.06952769999999),
  18: (-22.8229842, -47.0782628),
  20: (-22.8133258, -47.0691729),
  21: (-22.8193885, -47.06952769999999),
  23: (-22.821026, -47.072902)},
 {7: (-22.821026, -47.072902),
  8: (-22.8193885, -47.06952769999999),
  12: (-22.8186891, -47.0695881),
  13: (-22.8229842, -47.0782628),
  14: (-22.8133258, -47.0691729),
  17: (-22.808654, -47.0729041),
  18: (-22.8081856, -47.07795350000001),
  19: (-22.8100702, -47.07879519999999),
  20: (-22.8193885, -47.06952769999999),
  21: (-22.821026, -47.072902)},
 {7: (-22.821026, -47.072902),
  8: (-22.8193885, -47.06952769999999),
  9: (-22.821026, -47.072902),
  10: (-22.8229061, -47.0706183),
  11: (-22.8193885, -47.06952769999999),
  12: (-22.8228634, -47.07054),
  14: (-22.8133258, -47.0691729),
  15: (-22.810265, -47.0767833),
  16: (-22.8117485, -47.0792344),
  17: (-22.821026, -47.072902),
  18: (-

Taking the locations and turning them into a XML trips, which are written to PATHGEN.trips.xml

In [65]:
def parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_types_per_student, n_vtypes=5):
    # Creates the XML for the trips
    # The number of vtypes is the number of vTypes created for each style
    # The current supported styles are "agg" for aggressive and "norm" for normal
    importlib.reload(vehParameters)

    xml = '<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n'

    xml += '\n'
    xml += '<!-- Vehicles -->\n'
    vehdists = vehParameters.generateVehicleTypes(styles, n_vtypes)
    xml += vehParameters.parseVehiclesXML(vehdists, styles)

    xml += '\n'
    xml += '<!-- Trips -->\n'
    for i in range(len(location_time_list)):
        path, stop_durations = getPath(location_time_list[i], parkingAreas)
        xml += pathToXML(path, f'veh{i + 1}', veh_types_per_student[i], departure_times[i], stop_durations) + '\n'
    xml += '</routes>'
    with open('PATHGEN.trips.xml', 'w') as f:
        f.write(xml)
    return xml

In [66]:
departure_times = []
for i in range(len(location_time_list)):
    departure_times.append((list(location_time_list[i].keys())[0] - 7) * 20)

styles = ["agg", "norm"] # Currently only two styles, but can be expanded
n_vtypes = 5 
veh_style_per_student = []
for i in range(len(location_time_list)):
    if i < len(location_time_list) / 2:
        veh_style_per_student.append('agg')
    else: 
        veh_style_per_student.append('norm')   

# Write xml to file PATHGEN.trips.xml
xml = parseTripXML(location_time_list, parkingAreas, departure_times, styles, veh_style_per_student, n_vtypes)
print(xml)

<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">

<!-- Vehicles -->
<vTypeDistribution id="agg">
	<vType id="v_agg0" minGap="0.19" accel="3.67" decel="4.19" startupDelay="0.39" sigma="0.82" tau="1.2" maxSpeed="123.21" speedFactor="1.69" lcStrategic="8.0" lcCooperative="0.1" lcSpeedGain="5.28" lcKeepRight="0.57" lcOvertakeRight="0.64" lcSpeedGainLookahead="2.86" lcOvertakeDeltaSpeedFactor="0.78" lcPushy="0.61" lcAssertive="3.49" lcImpatience="0.77" lcTimeToImpatience="8.75" lcLaneDiscipline="1.39" lcSigma="0.83" lcAccelLat="2.01" probability="0.01596495294441153">
		<param key="device.rerouting.probability" value="1.0"/>
		<param key="device.rerouting.adaptation-steps" value="18"/>
		<param key="device.rerouting.adaptation-interval" value="10"/>
	</vType>
	<vType id="v_agg1" minGap="0.16" accel="3.51" decel="5.02" startupDelay="0.14" sigma="0.74" tau="1.4" maxSpeed="125.28" speedFactor="1.74" lcStrategi

Getting alternative routes for every route found

In [68]:
def getAltRoutes(net_path, trips_path, additional_path, iterations):
    # Run the duarouter to find alternative routes
    # Currently this only works for less than 10 iterations because of file naming
    for file in os.listdir('.'):
        if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
            os.remove(file)
    
    try:
        subprocess.run(['python3', duaiterate_path] + getArgs(net_path, trips_path, additional_path, iterations), check=True)
        shutil.move(f'00{iterations-1}/PATHGEN_00{iterations-1}.rou.alt.xml', f'PATHGEN_00{iterations-1}.rou.alt.xml')
        for i in range(iterations):
            if os.path.exists(f'00{i}'):
                shutil.rmtree(f'00{i}')
        for file in os.listdir('.'):
            if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
                os.rename(file, f'PATHGEN.rou.alt.xml')
    except subprocess.CalledProcessError as e:
        print(f"{e}")

In [69]:
getAltRoutes('osm.net.xml', 'PATHGEN.trips.xml', 'osm_stops.add.xml', 5)

> Executing step 0
>> Running router on PATHGEN.trips.xml
>>> Begin time: 2024-11-01 20:29:07.339982
>>> End time: 2024-11-01 20:29:08.163089
>>> Duration: 0:00:00.823107
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:08.163183
>>> End time: 2024-11-01 20:29:10.027174
>>> Duration: 0:00:01.863991
<<
< Step 0 ended (duration: 0:00:02.687891)
------------------

> Executing step 1
>> Running router on 000/PATHGEN_000.rou.alt.xml
>>> Begin time: 2024-11-01 20:29:10.033247
>>> End time: 2024-11-01 20:29:10.842358
>>> Duration: 0:00:00.809111
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:10.842498
>>> End time: 2024-11-01 20:29:12.644832
>>> Duration: 0:00:01.802334
<<
< Step 1 ended (duration: 0:00:02.611917)
------------------

> Executing step 2
>> Running router on 001/PATHGEN_001.rou.alt.xml
>>> Begin time: 2024-11-01 20:29:12.658072
>>> End time: 2024-11-01 20:29:13.441470
>>> Duration: 0:00:00.783398
<<
>> Running simulation
>>> Begin time: 2024-11-01 20:29:13.

To use this file, just set PATHGEN.rou.alt.xml as a trips file in osm.sumocfg